# DL. RuBERT

In [1]:
!pip install transformers -q
!pip install torch -q
!pip install scikit-learn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.8 MB/s eta 0:00:00


In [2]:
!pip install pymorphy3 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 57.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import time
from sklearn.metrics import classification_report, roc_auc_score
import warnings
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder


sklearn.set_config(transform_output="pandas")
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('labelled_data_1.csv')
df.shape

(10272, 13)

In [9]:
df = df[(~df.cluster.isna())&(df.cluster != '?')]

## Базовая предобработка

In [10]:
import pymorphy3
import re

morph = pymorphy3.MorphAnalyzer()

replacement_dict = {
    'provider': 'провайдер',
    'esim': 'есим',
    'b2b': 'б2б',
    'b2c': 'б2с',
    'invoice': 'инвойс',
    'tele2': 'теле2',
    'mts': 'мтс',
    'sim': 'сим',
    'sim-карта': 'сим-карта',
    'server': 'сервер',
    'sms': 'смс',
    'vimpelcom': 'вымпелком',
    'whatsapp': 'ватсапп',
    'message': 'сообщение',
    'access': 'доступ',
    'team': 'команда',
    'systems': 'система',
    'delivery': 'доставка',
    'diagnostic': 'диагностика',
    'information': 'информация',
    'help': 'помощь',
    'wd': 'вд',
    'ru': 'ру',
    'mobile': 'мобильный',
    'mailbox': 'мэилбокс',
    'sim-меню': 'сим-меню',
    'id': 'айди',
    'snminakovabeeline': 'билайн',
    'beeline': 'билайн',
    'iphone': 'айфон',
    'gprs': 'грпс',
    'ltmobidsupportmts': 'мтс',
    'team': 'команда',
    '-2024просьба': 'просьба',
    'эл': 'электронный'
}


def process_text(text):
    text = re.sub(r'\s*TT\d{8}', '', text)

    for eng_word, rus_word in replacement_dict.items():
        text = text.replace(eng_word.lower(), rus_word)

    if 'ЧС' in text:
        return 'черный список'
    if 'зарегистрировано обращение' in text.lower():
        return 'регистрация обращения'

    text = re.sub(r'[A-Za-z]', '', text)
    text = re.sub(r'[^\w\s-]', ' ', text)

    words = text.lower().split()
    normalized_words = [morph.parse(word)[0].normal_form for word in words if re.search(r'[а-яё]', word)]

    return ' '.join(normalized_words)

df['processed_theme'] = df['theme'].apply(process_text)
df[['theme', 'processed_theme']].head(20)

,theme,processed_theme
0,Жалоба абонента TT20595877,жалоба абонент
1,Жалоба абонента TT20592639,жалоба абонент
2,Жалоба абонента TT20594559,жалоба абонент
3,Жалоба абонента TT20593507,жалоба абонент
4,Жалоба абонента TT20593899,жалоба абонент
5,Re: [Ticket#10276498] Уведомление.Сбой на стор...,уведомление сбой на сторона оператор теле2-каз...
6,ЧС по 99993 от 01.01.2023,черный список
7,Жалоба абонента TT20595548,жалоба абонент
8,Жалоба абонента TT20598328,жалоба абонент
9,Жалоба абонента TT20591891,жалоба абонент


In [11]:
class RuBertCleaner:
    def __init__(self):
        self.phone_pattern = re.compile(r'^((8|\+7)[\- ]?)?(\(?\d{3}\)?[\- ]?)?[\d\- ]{7,10}$')
        self.name_pattern = re.compile(r'\b[А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+\b')
        self.sim_card_pattern = re.compile(r'\bсим-карт\w*\b')

    def clean_text(self, text):
        if not isinstance(text, str):
            return ''
        text = self.phone_pattern.sub('номер телефона', text)
        text = self.name_pattern.sub('фио', text)
        text = self.sim_card_pattern.sub('сим-карта', text)
        text = re.sub(r'(?<![\w])\d+(?![\w])', '', text)
        text = re.sub(r'\s-\s', ' ', text)
        text = re.sub(r'(?<!\S)-|-(?!\S)', '', text)
        return text.strip()

cleaner = RuBertCleaner()
df["text_clean"] = df["descr"].apply(cleaner.clean_text)

##  Применение pretrained модели

In [12]:
df_clean = df[df["cluster"] != "?"].copy()
df_clean["text"] = (
    df_clean["sender_name"].fillna('') + " " +
    df_clean["theme"].fillna('') + " " +
    df_clean["text_clean"].fillna('')
).str.strip()

label_encoder = LabelEncoder()
df_clean["label"] = label_encoder.fit_transform(df_clean["cluster"])

X_train, X_test, y_train, y_test = train_test_split(
    df_clean["text"].values,
    df_clean["label"].values,
    test_size=0.25,
    stratify=df_clean["label"].values,
    random_state=42
)

MODEL_NAME = "sberbank-ai/ruBert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_texts(texts):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

train_encodings = tokenize_texts(X_train)
test_encodings = tokenize_texts(X_test)


config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Помогла статья: https://habr.com/ru/articles/682476/

Создадим класс для нашего датасета, затем обучим модель.

In [13]:
class TicketDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = TicketDataset(train_encodings, y_train)
test_dataset = TicketDataset(test_encodings, y_test)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label_encoder.classes_)
)

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm
from sklearn.metrics import classification_report, roc_auc_score
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

epochs = 4
batch_size = 16
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
loss_fn = nn.CrossEntropyLoss()

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: train loss = {total_loss:.4f}")


Epoch 1: 100%|██████████| 473/473 [05:54<00:00,  1.33it/s]


Epoch 1: train loss = 175.2018


Epoch 2: 100%|██████████| 473/473 [05:57<00:00,  1.32it/s]


Epoch 2: train loss = 33.2817


Epoch 3: 100%|██████████| 473/473 [05:58<00:00,  1.32it/s]


Epoch 3: train loss = 16.1104


Epoch 4: 100%|██████████| 473/473 [05:58<00:00,  1.32it/s]

Epoch 4: train loss = 7.7835


In [16]:
model.eval()  # оценка
all_preds = []
all_labels = []
all_logits = []

start_time = time.time()


with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch["labels"].cpu().numpy())
        all_logits.extend(logits.cpu().numpy())

end_time = time.time()
total_time = end_time - start_time

print(f"\n Total inference time: {total_time:.2f} seconds")


 Total inference time: 34.10 seconds


In [18]:
import torch.nn.functional as F

all_probs = F.softmax(torch.tensor(all_logits), dim=1).numpy()

print(f"ROC AUC: {roc_auc_score(all_labels, all_probs, multi_class='ovr'):.4f}")

ROC AUC: 0.9992


In [17]:
print("\nclassification_report:")

target_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(all_labels, all_preds, target_names=target_names))


classification_report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00      1024
          10       0.98      0.96      0.97        93
           2       0.93      1.00      0.97        43
           3       0.98      0.98      0.98       107
           4       0.98      0.97      0.98       483
           5       0.94      0.95      0.95       123
           6       0.98      0.98      0.98       182
           7       0.95      0.98      0.96        83
           8       0.99      0.98      0.98        81
           9       0.99      1.00      1.00       299

    accuracy                           0.99      2518
   macro avg       0.97      0.98      0.98      2518
weighted avg       0.99      0.99      0.99      2518



In [19]:
model.save_pretrained("/content/rb_model")
tokenizer.save_pretrained("/content/rb_tokenizer")

('/content/rb_tokenizer/tokenizer_config.json',
 '/content/rb_tokenizer/special_tokens_map.json',
 '/content/rb_tokenizer/vocab.txt',
 '/content/rb_tokenizer/added_tokens.json',
 '/content/rb_tokenizer/tokenizer.json')

In [20]:
torch.save(model.state_dict(), "/content/tr_rb_model.pth")

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(label_encoder.classes_))
model.load_state_dict(torch.load("/content/tr_rb_model.pth"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Выводы

После бустингов, которые уже дали отличный результат, опробовали предобученный rubert.
Оценка моделей выполнена с помощью метрик ROC AUC, precision, recall, f1-score и accuracy.

- Logistic Regression: 0.908
- SVC: 0.935
- CatBoost: 0.984
- Optuna + CatBoost: 0.982
- XGBoost:  0.971
- Optuna + XGBoost: 0.968
- Lightgbm: 0.981
- Optuna + Lightgbm: 0.982
- RuBERT:  0.998 \
RuBERT показала наилучшее качество по метрике ROC AUC, немного превосходя бустинги.

Сравнение по метрике accuracy (доля правильных ответов)
- Logistic Regression: 0.70
- SVC: 0.75
- CatBoost: 0.88
- Optuna + CatBoost: 0.88
- XGBoost: 0.86
- Optuna + XGBoost: 0.86
- Lightgbm: 0.87
- Optuna + Lightgbm: 0.87
- RuBERT: 0.97\
RuBERT показала наивысшую точность.